In [1]:
!pip install sentencepiece

In [2]:
!pip install -q accelerate==0.27.2 peft==0.8.2 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 33.1 MB/s eta 0:00:00


In [3]:
!pip install 'transformers @ git+https://github.com/huggingface/transformers.git@831bc25d8fdb85768402f772cf65cc3d7872b211'

  Cloning https://github.com/huggingface/transformers.git (to revision 831bc25d8fdb85768402f772cf65cc3d7872b211) to /tmp/pip-install-twmftjfz/transformers_8bb1501e56d9490391a5113496d8218e
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-twmftjfz/transformers_8bb1501e56d9490391a5113496d8218e
  Running command git rev-parse -q --verify 'sha^831bc25d8fdb85768402f772cf65cc3d7872b211'
  Running command git fetch -q https://github.com/huggingface/transformers.git 831bc25d8fdb85768402f772cf65cc3d7872b211
  Running command git checkout -q 831bc25d8fdb85768402f772cf65cc3d7872b211
  Resolved https://github.com/huggingface/transformers.git to commit 831bc25d8fdb85768402f772cf65cc3d7872b211
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 46.9 MB/s eta 0:00:00
  Created wheel for transfo

In [4]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel

In [5]:
from huggingface_hub import login
login()

In [6]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"


################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [7]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
#tokenizer = AutoTokenizer.from_pretrained('mrm8488/phi-2-coder', use_fast=False)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
)

In [13]:
def generate_arabic_answer(question):

    # Generate response using the model
    generation = generator(question, max_length=150, num_return_sequences=1)

    # Extract the generated text and handle potential issues with generation
    if generation:
        answer = generation[0]['generated_text']
        print("Question:", question)
        print("Answer:", answer)
        return answer
    else:
        print("No valid answer was generated.")
        return None


In [14]:
# arabic_question = "ما هو أفضل كتاب لتعلم البرمجة؟"
# generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما هو أفضل كتاب لتعلم البرمجة؟
Answer: ما هو أفضل كتاب لتعلم البرمجة؟

أهلاً! هناك العديد من الكتب والمراجع التي يمكنك استخدامها لتعلم البرمجة، ولكنني سأقترح عليك بعض الكتب والمراجع التي تعتبر جيدة لتعلم البرمجة:

1. "Code Complete" by Steve McConnell - This book is a comprehensive guide to writing clean, maintainable code.
2. "Clean Code" by Robert C. Martin - This book is a must-read for any programmer, as it provides practical advice on writing clean, readable code.
3. "The Pragmatic Programmer" by Andrew Hunt and David Thomas - This book is a classic in the field of programming, and it


'ما هو أفضل كتاب لتعلم البرمجة؟\n\nأهلاً! هناك العديد من الكتب والمراجع التي يمكنك استخدامها لتعلم البرمجة، ولكنني سأقترح عليك بعض الكتب والمراجع التي تعتبر جيدة لتعلم البرمجة:\n\n1. "Code Complete" by Steve McConnell - This book is a comprehensive guide to writing clean, maintainable code.\n2. "Clean Code" by Robert C. Martin - This book is a must-read for any programmer, as it provides practical advice on writing clean, readable code.\n3. "The Pragmatic Programmer" by Andrew Hunt and David Thomas - This book is a classic in the field of programming, and it'

In [16]:
arabic_question ="ما هو تعريف الحضارة الرومانية؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما هو تعريف الحضارة الرومانية؟
Answer: ما هو تعريف الحضارة الرومانية؟

* 2000-2001: "الرومان في مصر" (The Romans in Egypt)
* 2001-2002: "الرومان في آسيا" (The Romans in Asia)
* 2002-2003: "الرومان في أفريقيا" (The Romans in Africa)
* 2003-2004: "الرومان في أوروبا" (The Romans in Europe)
* 2004-2005: "الرومان في البحر المتوسط" (The Romans in the Mediterranean Sea)
* 2005-2006: "الرومان في التاريخ" (The Romans in History)
* 200


'ما هو تعريف الحضارة الرومانية؟\n\n* 2000-2001: "الرومان في مصر" (The Romans in Egypt)\n* 2001-2002: "الرومان في آسيا" (The Romans in Asia)\n* 2002-2003: "الرومان في أفريقيا" (The Romans in Africa)\n* 2003-2004: "الرومان في أوروبا" (The Romans in Europe)\n* 2004-2005: "الرومان في البحر المتوسط" (The Romans in the Mediterranean Sea)\n* 2005-2006: "الرومان في التاريخ" (The Romans in History)\n* 200'

In [17]:
arabic_question ="ما هي أشهر الملوك والفراعنة في الحضارة المصرية القديمة؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما هي أشهر الملوك والفراعنة في الحضارة المصرية القديمة؟
Answer: ما هي أشهر الملوك والفراعنة في الحضارة المصرية القديمة؟

أشهر الملوك والفراعنة في الحضارة المصرية القديمة هم:

1. خوفو (2589-2566 ق.م): ملك مصر القديمة المعروف باسم خوفو الثاني، بني الممالك الثلاث في مصر القديمة، وتم بناء هرم خوفو في جيزة.
2. خعفر (2520-2494 ق.م): ملك مصر القديمة المعروف باسم خعفر الثاني، بني الممالك الثلاث في مصر القديمة، وتم بناء هرم


'ما هي أشهر الملوك والفراعنة في الحضارة المصرية القديمة؟\n\nأشهر الملوك والفراعنة في الحضارة المصرية القديمة هم:\n\n1. خوفو (2589-2566 ق.م): ملك مصر القديمة المعروف باسم خوفو الثاني، بني الممالك الثلاث في مصر القديمة، وتم بناء هرم خوفو في جيزة.\n2. خعفر (2520-2494 ق.م): ملك مصر القديمة المعروف باسم خعفر الثاني، بني الممالك الثلاث في مصر القديمة، وتم بناء هرم'

In [18]:
arabic_question ="وفاه توت عنخ امون ؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: وفاه توت عنخ امون ؟
Answer: وفاه توت عنخ امون ؟
كيف يمتلكها توت عنخ امون ؟
ما هو مصلح توت عنخ امون ؟
كيف يمتلكها توت عنخ امون ؟
ما هو مصلح توت عنخ امون ؟
كيف يمتلكها توت عنخ امون ؟
ما هو مصلح توت عنخ امون ؟
كيف يمتلكها توت عنخ امون ؟
ما هو مصلح توت عنخ امون ؟
كيف يمتلكها توت عنخ امون ؟
ما هو مصلح توت عنخ امون ؟
كيف يمتلك


'وفاه توت عنخ امون ؟\nكيف يمتلكها توت عنخ امون ؟\nما هو مصلح توت عنخ امون ؟\nكيف يمتلكها توت عنخ امون ؟\nما هو مصلح توت عنخ امون ؟\nكيف يمتلكها توت عنخ امون ؟\nما هو مصلح توت عنخ امون ؟\nكيف يمتلكها توت عنخ امون ؟\nما هو مصلح توت عنخ امون ؟\nكيف يمتلكها توت عنخ امون ؟\nما هو مصلح توت عنخ امون ؟\nكيف يمتلك'

In [19]:
arabic_question ="أين دفنت حتشبسوت؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: أين دفنت حتشبسوت؟
Answer: أين دفنت حتشبسوت؟

أما فين دفنت حتشبسوت؟

أما فين دفنت حتشبسوت؟

أما فين دفنت حتشبسوت؟

أما فين دفنت حتشبسوت؟

أما فين دفنت حتشبسوت؟

أما فين دفنت حتشبسوت؟

أما فين دفنت حتشبسوت؟

أما فين دفنت حتشبسوت؟

أما فين دفنت حتشبسوت؟

أما فين دفنت حتشبسوت؟

أما فين دفنت حتشبسوت؟

أما فين دفنت حتش


'أين دفنت حتشبسوت؟\n\nأما فين دفنت حتشبسوت؟\n\nأما فين دفنت حتشبسوت؟\n\nأما فين دفنت حتشبسوت؟\n\nأما فين دفنت حتشبسوت؟\n\nأما فين دفنت حتشبسوت؟\n\nأما فين دفنت حتشبسوت؟\n\nأما فين دفنت حتشبسوت؟\n\nأما فين دفنت حتشبسوت؟\n\nأما فين دفنت حتشبسوت؟\n\nأما فين دفنت حتشبسوت؟\n\nأما فين دفنت حتشبسوت؟\n\nأما فين دفنت حتش'

In [20]:
arabic_question ="ما سبب انتحار كليوباترا وانطونيوس؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما سبب انتحار كليوباترا وانطونيوس؟
Answer: ما سبب انتحار كليوباترا وانطونيوس؟" (Why did Cleopatra and Antony's suicide occur?)

* "النصائح والتنويمات في حياة كليوباترا" (The advice and teachings in Cleopatra's life)

* "كليوباترا والقهر" (Cleopatra and the oppression)

* "كليوباترا والشذوذ" (Cleopatra and the debauchery)

* "كليوباترا والجلالة" (Cleopatra and the majesty)

* "كليوباترا والهوية" (Cleopatra and the passion)

* "


'ما سبب انتحار كليوباترا وانطونيوس؟" (Why did Cleopatra and Antony\'s suicide occur?)\n\n* "النصائح والتنويمات في حياة كليوباترا" (The advice and teachings in Cleopatra\'s life)\n\n* "كليوباترا والقهر" (Cleopatra and the oppression)\n\n* "كليوباترا والشذوذ" (Cleopatra and the debauchery)\n\n* "كليوباترا والجلالة" (Cleopatra and the majesty)\n\n* "كليوباترا والهوية" (Cleopatra and the passion)\n\n* "'

In [21]:
arabic_question ="ما هو الغرض من بناء هرم خوفو؟ "
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما هو الغرض من بناء هرم خوفو؟ 
Answer: ما هو الغرض من بناء هرم خوفو؟  |

### 2009

* «أهلاً» (Ahlan) - "Welcome" (with Mohamed Mounir)
* «النار» (An-Nar) - "The Fire"
* «أيها» (Ayha) - "Oh, You"
* «أيها» (Ayha) - "Oh, You" (Remix)
* «أهلاً» (Ahlan) - "Welcome" (Remix)

### 2010

* «ما هو الغرور» (Mā Huwa Al-Ghuroor) - "What is Pride"
* «أيها» (


'ما هو الغرض من بناء هرم خوفو؟  |\n\n### 2009\n\n* «أهلاً» (Ahlan) - "Welcome" (with Mohamed Mounir)\n* «النار» (An-Nar) - "The Fire"\n* «أيها» (Ayha) - "Oh, You"\n* «أيها» (Ayha) - "Oh, You" (Remix)\n* «أهلاً» (Ahlan) - "Welcome" (Remix)\n\n### 2010\n\n* «ما هو الغرور» (Mā Huwa Al-Ghuroor) - "What is Pride"\n* «أيها» ('

In [22]:
arabic_question ="ما هو ارتفاع هرم منقرع؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما هو ارتفاع هرم منقرع؟
Answer: ما هو ارتفاع هرم منقرع؟

أجوبة:

1. 1,440 متر (4,724 قدم) - ارتفاع هرم خوفو في مصر.
2. 246 متر (804 قدم) - ارتفاع هرم خوفو في مصر.
3. 139 متر (456 قدم) - ارتفاع هرم خوفو في مصر.
4. 100 متر (328 قدم) - ارتفاع هرم خوفو في مصر.

الجواب الصحيح هو 1. 1,440 متر (4,724 قدم) - ارتفاع هرم خوفو في مصر. هرم خوفو هو أحد أهرامات مصر القديمة، ويبلغ ارتفاعه 1,440 متر (


'ما هو ارتفاع هرم منقرع؟\n\nأجوبة:\n\n1. 1,440 متر (4,724 قدم) - ارتفاع هرم خوفو في مصر.\n2. 246 متر (804 قدم) - ارتفاع هرم خوفو في مصر.\n3. 139 متر (456 قدم) - ارتفاع هرم خوفو في مصر.\n4. 100 متر (328 قدم) - ارتفاع هرم خوفو في مصر.\n\nالجواب الصحيح هو 1. 1,440 متر (4,724 قدم) - ارتفاع هرم خوفو في مصر. هرم خوفو هو أحد أهرامات مصر القديمة، ويبلغ ارتفاعه 1,440 متر ('

In [23]:
arabic_question ="كيف أثر محمد علي باشا على التنوع الثقافي في مصر؟"
generate_arabic_answer(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: كيف أثر محمد علي باشا على التنوع الثقافي في مصر؟
Answer: كيف أثر محمد علي باشا على التنوع الثقافي في مصر؟

محمد علي باشا، الذي تولى الحكم في مصر عام 1805، كان له أثر كبير في التنوع الثقافي في مصر. من أهم ما أثر به هو:

1. إحياء الفنون والفنون: محمد علي باشا كان معجباً بالفنون والفنون، فدعمها والفنانين، مما أدى إلى إحياء الفنون والفنون في مصر، مثل الفنون الإسلامية والكنائس والقصور والمنارات.
2. دعم التعليم: محمد علي باشا دعم التعليم في مصر، مما أدى إلى إنشاء


'كيف أثر محمد علي باشا على التنوع الثقافي في مصر؟\n\nمحمد علي باشا، الذي تولى الحكم في مصر عام 1805، كان له أثر كبير في التنوع الثقافي في مصر. من أهم ما أثر به هو:\n\n1. إحياء الفنون والفنون: محمد علي باشا كان معجباً بالفنون والفنون، فدعمها والفنانين، مما أدى إلى إحياء الفنون والفنون في مصر، مثل الفنون الإسلامية والكنائس والقصور والمنارات.\n2. دعم التعليم: محمد علي باشا دعم التعليم في مصر، مما أدى إلى إنشاء'

In [24]:
arabic_question ="كم عدد انواع الرهبانية؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: كم عدد انواع الرهبانية؟
Answer: كم عدد انواع الرهبانية؟

النصيح: فيما هو منسجم أن هناك أشكال متعددة من الرهبانية، من بينها:

1. رهبانية المسيح: هي رهبانية مسيحية تقوم على حياة رهبانية كاثوليكية، ويتمتعون بالعديد من الصلوات والصوم والفضائل.
2. رهبانية البوذية: هي رهبانية بوذية تقوم على حياة رهبانية بوذية، ويتمتعون بالعديد من الصلوات والصوم والفضائل.
3. رهبانية الهندوسية: هي رهبانية هند


'كم عدد انواع الرهبانية؟\n\nالنصيح: فيما هو منسجم أن هناك أشكال متعددة من الرهبانية، من بينها:\n\n1. رهبانية المسيح: هي رهبانية مسيحية تقوم على حياة رهبانية كاثوليكية، ويتمتعون بالعديد من الصلوات والصوم والفضائل.\n2. رهبانية البوذية: هي رهبانية بوذية تقوم على حياة رهبانية بوذية، ويتمتعون بالعديد من الصلوات والصوم والفضائل.\n3. رهبانية الهندوسية: هي رهبانية هند'

In [25]:
arabic_question ="في أي عام تولى محمد علي باشا الحكم في مصر؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: في أي عام تولى محمد علي باشا الحكم في مصر؟
Answer: في أي عام تولى محمد علي باشا الحكم في مصر؟

أجابة: تولى محمد علي باشا الحكم في مصر في عام 1805.

س: ما هو اسم القائد العسكري الذي قاد قواته إلى النصر في معركة أبي قير ضد القوات الروسية في عام 1798؟

أجابة: اسم القائد العسكري هو حسين رشدي باشا.

س: ما هو اسم القائد العسكري الذي قاد قواته إلى النصر في معركة راملي في عام 1798؟

أجابة: اسم القائد العسكري هو يوسف سعد باشا.

س: ما هو اسم القائد


'في أي عام تولى محمد علي باشا الحكم في مصر؟\n\nأجابة: تولى محمد علي باشا الحكم في مصر في عام 1805.\n\nس: ما هو اسم القائد العسكري الذي قاد قواته إلى النصر في معركة أبي قير ضد القوات الروسية في عام 1798؟\n\nأجابة: اسم القائد العسكري هو حسين رشدي باشا.\n\nس: ما هو اسم القائد العسكري الذي قاد قواته إلى النصر في معركة راملي في عام 1798؟\n\nأجابة: اسم القائد العسكري هو يوسف سعد باشا.\n\nس: ما هو اسم القائد'

In [26]:
arabic_question ="ما هي خصائص الحضارة الرومانية؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما هي خصائص الحضارة الرومانية؟
Answer: ما هي خصائص الحضارة الرومانية؟

نقدم لكم في هذا المقال إجابةً على السؤال المذكور.

تعد الحضارة الرومانية من أهم الحضارات في التاريخ البشري، حيث أنها ساهمت في التطوير والازدهار في العديد من المجالات، مثل الهندسة والفلسفة والفنون واللغة والقانون، وغيرها. وهي تتمتع بالخصائص التالية:

1. التماسك والوحدة: تعتبر الحضارة الرومانية من أهم الحضارات التي ساهمت في التماسك والوحدة في أوروبا، حيث أنها ساهمت في


'ما هي خصائص الحضارة الرومانية؟\n\nنقدم لكم في هذا المقال إجابةً على السؤال المذكور.\n\nتعد الحضارة الرومانية من أهم الحضارات في التاريخ البشري، حيث أنها ساهمت في التطوير والازدهار في العديد من المجالات، مثل الهندسة والفلسفة والفنون واللغة والقانون، وغيرها. وهي تتمتع بالخصائص التالية:\n\n1. التماسك والوحدة: تعتبر الحضارة الرومانية من أهم الحضارات التي ساهمت في التماسك والوحدة في أوروبا، حيث أنها ساهمت في'

In [27]:
arabic_question ="متى تولى بطليموس الثالث حكم مصر؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: متى تولى بطليموس الثالث حكم مصر؟
Answer: متى تولى بطليموس الثالث حكم مصر؟

الجواب: تولى بطليموس الثالث حكم مصر في 51 ق.م. بعد أن قتل الملكة كليوباترا السابعة زوجته، وتمكن من الحصول على السلطة.

ما هو اسم الملكة التي قتلت بطليموس الثالث؟

الجواب: اسم الملكة التي قتلت بطليموس الثالث هو كليوباترا السابعة.

ما هو اسم الملك الذي خلف بطليموس الثالث في الحكم؟

الجواب: لم يكن هناك ملك يخلف بطليموس الثالث في الحكم، لأن بطليموس الثالث قتل في 51 ق.م. وتمكن من الح


'متى تولى بطليموس الثالث حكم مصر؟\n\nالجواب: تولى بطليموس الثالث حكم مصر في 51 ق.م. بعد أن قتل الملكة كليوباترا السابعة زوجته، وتمكن من الحصول على السلطة.\n\nما هو اسم الملكة التي قتلت بطليموس الثالث؟\n\nالجواب: اسم الملكة التي قتلت بطليموس الثالث هو كليوباترا السابعة.\n\nما هو اسم الملك الذي خلف بطليموس الثالث في الحكم؟\n\nالجواب: لم يكن هناك ملك يخلف بطليموس الثالث في الحكم، لأن بطليموس الثالث قتل في 51 ق.م. وتمكن من الح'

In [28]:
arabic_question ="كيف ساهم محمد علي باشا في تعزيز التقدم العلمي والتكنولوجي في مصر؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: كيف ساهم محمد علي باشا في تعزيز التقدم العلمي والتكنولوجي في مصر؟
Answer: كيف ساهم محمد علي باشا في تعزيز التقدم العلمي والتكنولوجي في مصر؟

محمد علي باشا ساهم في تعزيز التقدم العلمي والتكنولوجي في مصر من خلال إصدار عدد من القرارات والبرامج والهيئيات والمراكز والمنشآت والبرامج والهيئيات والمراكز والمنشآت والبرامج والهيئيات والمراكز والمنشآت والبرامج والهيئيات والمراكز والمنشآت والبرامج والهيئيات والمراكز والمنشآت والبرامج والهيئيات والمراكز والمن


'كيف ساهم محمد علي باشا في تعزيز التقدم العلمي والتكنولوجي في مصر؟\n\nمحمد علي باشا ساهم في تعزيز التقدم العلمي والتكنولوجي في مصر من خلال إصدار عدد من القرارات والبرامج والهيئيات والمراكز والمنشآت والبرامج والهيئيات والمراكز والمنشآت والبرامج والهيئيات والمراكز والمنشآت والبرامج والهيئيات والمراكز والمنشآت والبرامج والهيئيات والمراكز والمنشآت والبرامج والهيئيات والمراكز والمن'

In [29]:
arabic_question ="من قام ببناء مكتبة الإسكندرية؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: من قام ببناء مكتبة الإسكندرية؟
Answer: من قام ببناء مكتبة الإسكندرية؟

* الإسكندر الأكبر قام ببناء مكتبة الإسكندرية في القرن الرابع قبل الميلاد.

## مراجع

* "Alexandria, the Library of". Encyclopedia Britannica. 2016. Retrieved 2021-09-19.
* "The Library of Alexandria". Ancient History Encyclopedia. 2021. Retrieved 2021-09-19.
* "The Library of Alexandria". The Metropolitan Museum of Art. 2021. Retrieved 2021-09-19.

* بوابة مصر القديمة
* بوابة كتب
* بوابة علم المعلومات
* بوابة التاريخ
* بوابة اليونان القديم

* بوابة كتب


'من قام ببناء مكتبة الإسكندرية؟\n\n* الإسكندر الأكبر قام ببناء مكتبة الإسكندرية في القرن الرابع قبل الميلاد.\n\n## مراجع\n\n* "Alexandria, the Library of". Encyclopedia Britannica. 2016. Retrieved 2021-09-19.\n* "The Library of Alexandria". Ancient History Encyclopedia. 2021. Retrieved 2021-09-19.\n* "The Library of Alexandria". The Metropolitan Museum of Art. 2021. Retrieved 2021-09-19.\n\n* بوابة مصر القديمة\n* بوابة كتب\n* بوابة علم المعلومات\n* بوابة التاريخ\n* بوابة اليونان القديم\n\n* بوابة كتب'

In [30]:
arabic_question ="متى جاء الفتح الاسلامى على مصر ؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: متى جاء الفتح الاسلامى على مصر ؟
Answer: متى جاء الفتح الاسلامى على مصر ؟
I hope you understand the question. If not, please let me know and I'll try to rephrase it.

The question is: What is the historical context of the spread of Islam in Egypt? In other words, what were the circumstances that led to the adoption of Islam by the Egyptians in the 7th century?

To answer this, we need to look at the historical context of the time. In the 7th century, Egypt was a province of the Byzantine Empire, and the majority of its population was Christian. However, the Byzantine Empire was facing internal conflicts and external threats, which weakened its grip on the province.

Meanwhile, the Arabs, who were


"متى جاء الفتح الاسلامى على مصر ؟\nI hope you understand the question. If not, please let me know and I'll try to rephrase it.\n\nThe question is: What is the historical context of the spread of Islam in Egypt? In other words, what were the circumstances that led to the adoption of Islam by the Egyptians in the 7th century?\n\nTo answer this, we need to look at the historical context of the time. In the 7th century, Egypt was a province of the Byzantine Empire, and the majority of its population was Christian. However, the Byzantine Empire was facing internal conflicts and external threats, which weakened its grip on the province.\n\nMeanwhile, the Arabs, who were"

In [31]:
arabic_question ="من هم أشهر وأهم الفراعنة في الحضارة المصرية القديمة؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: من هم أشهر وأهم الفراعنة في الحضارة المصرية القديمة؟
Answer: من هم أشهر وأهم الفراعنة في الحضارة المصرية القديمة؟

1. فرعون سقنة: هو فرعون آخر من سلالة سقنة، وكان حكمه من 2589-2566 ق.م.
2. فرعون خوفو: هو فرعون من سلالة سقنة، وكان حكمه من 2520-2494 ق.م.
3. فرعون خفرع: هو فرعون من سلالة سقنة، وكان حكمه من 2494-2484 ق.م.
4. فرعون مينا: هو فرعون من سلالة سقنة،


'من هم أشهر وأهم الفراعنة في الحضارة المصرية القديمة؟\n\n1. فرعون سقنة: هو فرعون آخر من سلالة سقنة، وكان حكمه من 2589-2566 ق.م.\n2. فرعون خوفو: هو فرعون من سلالة سقنة، وكان حكمه من 2520-2494 ق.م.\n3. فرعون خفرع: هو فرعون من سلالة سقنة، وكان حكمه من 2494-2484 ق.م.\n4. فرعون مينا: هو فرعون من سلالة سقنة،'

In [32]:
arabic_question ="متى حكمت حتشبسوت مصر؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: متى حكمت حتشبسوت مصر؟
Answer: متى حكمت حتشبسوت مصر؟" (What is the meaning of the phrase "Egypt is a land of the Pharaohs"?) (pp. 21-22). In this context, the author is highlighting the significance of the Pharaonic era in Egyptian history and its impact on the country's identity.

The book also explores the concept of "Egyptian identity" and its relationship with the country's history and culture. The author argues that the concept of Egyptian identity is complex and multifaceted, and that it is shaped by various factors, including the country's ancient history, its Islamic heritage, and its modern-day experiences. The author also highlights the importance of the concept of "Egyptian-ness" in


'متى حكمت حتشبسوت مصر؟" (What is the meaning of the phrase "Egypt is a land of the Pharaohs"?) (pp. 21-22). In this context, the author is highlighting the significance of the Pharaonic era in Egyptian history and its impact on the country\'s identity.\n\nThe book also explores the concept of "Egyptian identity" and its relationship with the country\'s history and culture. The author argues that the concept of Egyptian identity is complex and multifaceted, and that it is shaped by various factors, including the country\'s ancient history, its Islamic heritage, and its modern-day experiences. The author also highlights the importance of the concept of "Egyptian-ness" in'

In [33]:
arabic_question ="كم كان عدد الحجارة المستخدمة في بناء هرم خوفو؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: كم كان عدد الحجارة المستخدمة في بناء هرم خوفو؟
Answer: كم كان عدد الحجارة المستخدمة في بناء هرم خوفو؟

يستخدم الباحثون أرقام مختلفة لوصف عدد الحجارة المستخدمة في بناء هرم خوفو. بعض الأرقام المقدرة هي:

* 2.3 مليون حجر صلب (وفقاً لدراسة عام 1994)
* 2.5 مليون حجر صلب (وفقاً لدراسة عام 2006)
* 3 مليون حجر صلب (وفقاً لدراسة عام 2010)
* 3.5 مليون حجر صلب (وفقاً لدراسة عام 201


'كم كان عدد الحجارة المستخدمة في بناء هرم خوفو؟\n\nيستخدم الباحثون أرقام مختلفة لوصف عدد الحجارة المستخدمة في بناء هرم خوفو. بعض الأرقام المقدرة هي:\n\n* 2.3 مليون حجر صلب (وفقاً لدراسة عام 1994)\n* 2.5 مليون حجر صلب (وفقاً لدراسة عام 2006)\n* 3 مليون حجر صلب (وفقاً لدراسة عام 2010)\n* 3.5 مليون حجر صلب (وفقاً لدراسة عام 201'

In [34]:
arabic_question ="كيف تعامل محمد علي باشا مع القضايا المتعلقة بالتخطيط العمراني في مصر؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: كيف تعامل محمد علي باشا مع القضايا المتعلقة بالتخطيط العمراني في مصر؟
Answer: كيف تعامل محمد علي باشا مع القضايا المتعلقة بالتخطيط العمراني في مصر؟

محمد علي باشا هو واحد من أهم القضاة الذين ساهموا في التخطيط العمراني في مصر خلال القرن التاسع عشر. خلال فترة ولايته (1805-1848)، قام محمد علي باشا بتطوير عدد من المشروعات والبرامج التي هدفها إلى تحسين حالة التخطيط العمراني في مصر.

من أهم ما قام به محمد علي باشا في مجال التخطيط العمراني في مصر هو:

1. تطوير مدينة القاهرة: قام محمد علي باشا بتطوير مدينة القاهرة، التي


'كيف تعامل محمد علي باشا مع القضايا المتعلقة بالتخطيط العمراني في مصر؟\n\nمحمد علي باشا هو واحد من أهم القضاة الذين ساهموا في التخطيط العمراني في مصر خلال القرن التاسع عشر. خلال فترة ولايته (1805-1848)، قام محمد علي باشا بتطوير عدد من المشروعات والبرامج التي هدفها إلى تحسين حالة التخطيط العمراني في مصر.\n\nمن أهم ما قام به محمد علي باشا في مجال التخطيط العمراني في مصر هو:\n\n1. تطوير مدينة القاهرة: قام محمد علي باشا بتطوير مدينة القاهرة، التي'

In [35]:
arabic_question ="ما هي الإنجازات الثقافية والفنية المرتبطة بعصر محمد علي باشا؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما هي الإنجازات الثقافية والفنية المرتبطة بعصر محمد علي باشا؟
Answer: ما هي الإنجازات الثقافية والفنية المرتبطة بعصر محمد علي باشا؟

1. تأسيس المدارس والجامعات والهيئات الثقافية والفنية في مصر.
2. تأسيس صحيفة «الوافي» ل пропاغدة الثقافية والفنية.
3. تأسيس المسرح المصرى والهيئات الموسيقية والهيئات المسرحية.
4. دعوة الفنانين والشعراء والكتاب والملحنين والملحينيات إلى مصر لتعزيز الثقافة والفنون.
5. تأسيس مهرجان «مهرجان القاهرة» ل


'ما هي الإنجازات الثقافية والفنية المرتبطة بعصر محمد علي باشا؟\n\n1. تأسيس المدارس والجامعات والهيئات الثقافية والفنية في مصر.\n2. تأسيس صحيفة «الوافي» ل пропاغدة الثقافية والفنية.\n3. تأسيس المسرح المصرى والهيئات الموسيقية والهيئات المسرحية.\n4. دعوة الفنانين والشعراء والكتاب والملحنين والملحينيات إلى مصر لتعزيز الثقافة والفنون.\n5. تأسيس مهرجان «مهرجان القاهرة» ل'

In [36]:
arabic_question ="ما هي الابتكارات العسكرية التي أدخلها محمد علي باشا؟"
generate_arabic_answer(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما هي الابتكارات العسكرية التي أدخلها محمد علي باشا؟
Answer: ما هي الابتكارات العسكرية التي أدخلها محمد علي باشا؟

أما الابتكارات العسكرية التي أدخلها محمد علي باشا في مصر فهي كالآتي:

1. إقامة المدارس العسكرية: قام محمد علي باشا بإقامة مدارس عسكرية في مصر لتدريب الجنود علىlatest التقنيات العسكرية والقتال.
2. توفير الأسلحة والذخائر: قام محمد علي باشا بتوفير الأسلحة والذخائر الحديثة لجيش مصر، مما ساعده في تحقيق النجاح في القتال.
3. إق


'ما هي الابتكارات العسكرية التي أدخلها محمد علي باشا؟\n\nأما الابتكارات العسكرية التي أدخلها محمد علي باشا في مصر فهي كالآتي:\n\n1. إقامة المدارس العسكرية: قام محمد علي باشا بإقامة مدارس عسكرية في مصر لتدريب الجنود علىlatest التقنيات العسكرية والقتال.\n2. توفير الأسلحة والذخائر: قام محمد علي باشا بتوفير الأسلحة والذخائر الحديثة لجيش مصر، مما ساعده في تحقيق النجاح في القتال.\n3. إق'